In [1]:
import sqlite3
import requests
import string
import datetime

# Settings 

In [2]:
#API root and auth
sale_id_api = 'https://logistics-api-dot-active-thunder-329100.rj.r.appspot.com'
chip_api = 'https://telecom-api-dot-active-thunder-329100.rj.r.appspot.com'
conversations_api = 'https://chats-api-dot-active-thunder-329100.rj.r.appspot.com'
header = {'authorization': 'teste'} #auth code

#HTTP Requests
#   Sale API
HTTP_request_sale_track = '/tracking'
HTTP_request_sale_zip = '/zip_code'
#   Chip Api
HTTP_request_chip = '/chip_status'
#   Conversation API
HTTP_request_conversation = '/conversations'
HTTP_request_conversation_info = '/conversation_info'

#Customer contact info
contact_info = 'customer_service@help.br'

# SQL tools

In [3]:
def get_sale_id_SQL(merchant_id):

    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT id_sale
        FROM sales
        WHERE sales.merchant_id = ?
    """
    c.execute(query, (merchant_id,))
    sale_id = c.fetchone()
    return {"id_sale": str(sale_id[0])} #gives a tupple, with a second null arg

In [4]:
#GET receipt info from merchant_id

def get_receipt_info_SQL(merchant_id):

    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT status, description, value, created_at
        FROM receipt
        WHERE receipt.merchant_id = ?
    """
    c.execute(query, (merchant_id,))
    receipt_info = c.fetchone()

    #variables
    status = receipt_info[0]
    description = receipt_info[1]
    value = receipt_info[2]
    created_at = receipt_info[3]
    
    created_at = created_at.split()[0] #Select only the date
    
    answ_dict = {'status':status, 'description': description, 'value': value, 'created_at': created_at }
    return answ_dict

In [5]:
#GET chip status from merchant_id
def get_chip_id_SQL(merchant_id):

    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT chip_id
        FROM sales
        WHERE sales.merchant_id = ?
    """
    c.execute(query, (merchant_id,))
    chip_id = c.fetchone()
    return {"chip_id": str(chip_id[0])} #gives a tupple, with a second null arg

In [6]:
#GET transactions from merchant_id

def get_transactions_info_SQL(merchant_id):

    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT transaction_id, value
        FROM transactions
        WHERE transactions.merchant_id = ?
    """
    c.execute(query, (merchant_id,))
    transactions_info = c.fetchall()

    transactions_info_list = []
    for transaction in transactions_info:
        
        transaction = list(transaction)
        transaction_dict = {'transaction_id': transaction[0], 'value': transaction[1]}
        transactions_info_list.append(transaction_dict)
        
    return transactions_info_list

In [7]:
#GET aggregated transactions from merchant_id

def get_transactions_aggregate_SQL(merchant_id): #for now initialize to first act conv

    #/!\ SQL Hardcoded Implement Cloud SQL on GCP
    conn = sqlite3.connect('/home/ecapi/code/e-capi/Chat_bot_Software_Engineer/Chat_bot_Software_Engineer/data/simulation-db')
    c = conn.cursor()
    query = """
        SELECT COUNT(transaction_id) as total_transactions, SUM(value) as total_value
        FROM transactions
        WHERE transactions.merchant_id = ?
    """
    c.execute(query, (merchant_id,))
    transactions_aggregation = c.fetchone()

    #variables
    transactions_amount = transactions_aggregation[0]
    transactions_value = transactions_aggregation[1]


    answ_dict = {'transactions_amount':transactions_amount, 'transactions_value': transactions_value }
    return answ_dict

# API tools

In [8]:
def generate_sale_tracking_info(params_request):
    contact_info = 'customer_service@help.br'
    url = sale_id_api+HTTP_request_sale_track

    try:
        response = requests.post(url, json=params_request, headers=header)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return f"An Http Error occurred: Please, contact us at: {contact_info}"
    except requests.exceptions.ConnectionError as errc:
        return f"A Connection Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.Timeout as errt:
        return f"A Timeout Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.RequestException as err:
        return f"An Unknown Error occurred: Please, contact us at: {contact_info}"

    return response.json()

In [9]:
#Zip code Function

def generate_sale_zip_info(params_request):
    contact_info = 'customer_service@help.br'
    url = sale_id_api+HTTP_request_sale_zip

    try:
        response = requests.post(url, json=params_request, headers=header)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return f"An Http Error occurred: Please, contact us at: {contact_info}"
    except requests.exceptions.ConnectionError as errc:
        return f"A Connection Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.Timeout as errt:
        return f"A Timeout Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.RequestException as err:
        return f"An Unknown Error occurred: Please, contact us at: {contact_info}"

    return response.json()

In [10]:
#----Function Chip Status-----
def generate_chip_info(params_request):
    contact_info = 'customer_service@help.br'
    url = chip_api+HTTP_request_chip

    try:
        response = requests.post(url, json=params_request, headers=header)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return f"An Http Error occurred: Please, contact us at: {contact_info}"
    except requests.exceptions.ConnectionError as errc:
        return f"A Connection Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.Timeout as errt:
        return f"A Timeout Error occurred: Please, try again later or contact us at: {contact_info}"
    except requests.exceptions.RequestException as err:
        return f"An Unknown Error occurred: Please, contact us at: {contact_info}"

    return response.json()

In [11]:
#----Function all Active Conversations info----
def generate_all_act_conv_info():
    url = conversations_api+HTTP_request_conversation
    response = requests.post(url, headers=header)
    json_response = response.json() #retrieve active conversations ID

    active_chat_list = [] #Create a list of active conversations info
    for values in json_response.values():
        for val in values:
            params_conver_info = {'conversation_id': str(val)}
            response = requests.post(conversations_api+HTTP_request_conversation_info, headers=header, json=params_conver_info)
            active_chat_list.append(response.json())

        return active_chat_list

In [12]:
#----Function all Active Chats List----
def generate_act_convs_list():
    url = conversations_api+HTTP_request_conversation
    response = requests.post(url, headers=header)
    json_response = response.json() #retrieve active conversations ID

    active_convs_list = json_response.get('conversation_ids')
    return active_convs_list

In [13]:
#----Function one conversation info----
def generate_act_conv_info(conversation_id):

    url = conversations_api+HTTP_request_conversation_info
    params = {"conversation_id": conversation_id}
    response_conversation = requests.post(url, json=params, headers=header)
    conversation_info = response_conversation.json()

    return conversation_info

In [14]:
# for pipeline response
def generate_info_of_conv(conversation_id, info_needed):

    conversation_id = str(conversation_id)
    info_needed = str(info_needed)

    url = conversations_api+HTTP_request_conversation_info
    params = {"conversation_id": conversation_id}
    response = requests.post(url, json=params, headers=header)
    json_response = response.json()

    info = json_response.get(info_needed)

    return info

In [15]:
#Check the status for delivery responses
def delivery_api_status(param, code):

    code = str(code)
    param = str(param)

    if code == 'zip_code':
        response = generate_sale_zip_info({code: param})
    elif code == 'id_sale':
        response = generate_sale_tracking_info({code: param})

    if isinstance(response,dict):

        return 'ok'

    else:
        return 'error'

# Chat_tools

In [16]:
def responses_pipeline():
    active_chats = generate_act_convs_list()
    list_of_problems = []

    for conversation_id in list(active_chats):

        #give info for every all messages
        messages = generate_info_of_conv(conversation_id, 'messages') #messages of one conversation

        for message in list(messages):


            merchant_id = generate_info_of_conv(conversation_id,'merchant_id')
            subject = generate_info_of_conv(conversation_id, 'subject')

            features_pipeline = {'conversation_id': conversation_id, 'merchant_id': merchant_id, 'subject': subject}

            #create a list to parse on chatbot.py
            list_of_problems.append(features_pipeline)
            messages.remove(message)

        # {'conversation_id': chat_id, 'merchant_id':}
    return list_of_problems

In [17]:
responses_pipeline()

[{'conversation_id': 754893,
  'merchant_id': '558392',
  'subject': 'bank account receipt problem'},
 {'conversation_id': 754894,
  'merchant_id': '530364',
  'subject': 'connection problem'},
 {'conversation_id': 754895,
  'merchant_id': '384923',
  'subject': 'late delivery'},
 {'conversation_id': 754896,
  'merchant_id': '395023',
  'subject': 'check delivery address'},
 {'conversation_id': 754897,
  'merchant_id': '849583',
  'subject': 'delivery forecast'}]

In [18]:
get_receipt_info_SQL('558392')

{'status': 'failed',
 'description': "bank's API is unavailable",
 'value': 177.4,
 'created_at': '2021-12-22'}

In [19]:
get_transactions_info_SQL('558392')

[{'transaction_id': 374290, 'value': 127.4},
 {'transaction_id': 374289, 'value': 50.0}]

In [20]:
get_transactions_aggregate_SQL('558392')

{'transactions_amount': 2, 'transactions_value': 177.4}

In [21]:
def receipt_response(merchant_id):
    
    #get SQL receipt info
    try:
        total_receipt_info = get_receipt_info_SQL(merchant_id)
        total_transactions = get_transactions_aggregate_SQL(merchant_id).get('transactions_amount') #string
        transactions_info = get_transactions_info_SQL(merchant_id)
    except:
        return None
    
    #variables
    status = total_receipt_info.get('status')
    description = total_receipt_info.get('description')
    total_value = total_receipt_info.get('value')
    date = total_receipt_info.get('created_at')
    
    return f"\033[1mFirst of all we would like to remind you that transactions have a 24h delay before being transfered.\033[0m\nConcerning your demand here is some information:\nTransactions Date: \033[1m{date}\033[0m\nNumber of Transactions: \033[1m{total_transactions}\033[0m\nTransactions Total: \033[1mR${total_value}\033[0m\nStatus: \033[1m{status}\033[0m\nDescription: \033[1m{description}\033[0m\nTransactions Involved:\033[1m{transactions_info}\033[0m"

In [22]:
receipt_response('558392')

"\x1bFirst of all we would like to remind you that transactions have a 24h delay before being transfered.\x1b\nConcerning your demand here is some information:\nTransactions Date: \x1b2021-12-22\x1b\nNumber of Transactions: \x1b2\x1b\nTransactions Total: \x1bR$177.4\x1b\nStatus: \x1bfailed\x1b\nDescription: \x1bbank's API is unavailable\x1b\nTransactions Involved:\x1b[{'transaction_id': 374290, 'value': 127.4}, {'transaction_id': 374289, 'value': 50.0}]\x1b"

# Connection response

In [27]:
responses_pipeline()

[{'conversation_id': 754893,
  'merchant_id': '558392',
  'subject': 'bank account receipt problem'},
 {'conversation_id': 754894,
  'merchant_id': '530364',
  'subject': 'connection problem'},
 {'conversation_id': 754895,
  'merchant_id': '384923',
  'subject': 'late delivery'},
 {'conversation_id': 754896,
  'merchant_id': '395023',
  'subject': 'check delivery address'},
 {'conversation_id': 754897,
  'merchant_id': '849583',
  'subject': 'delivery forecast'}]

In [37]:
_chip = get_chip_id_SQL('530364')

In [38]:
generate_chip_info(_chip)

{'id': 'CHIP37649',
 'status': 'inactive',
 'description': 'Chip disabled due to non-use'}

In [45]:
def connection_response(merchant_id):
    #get SQL receipt info
    try:
        chip_id_SQL = get_chip_id_SQL(merchant_id)
        chip_info = generate_chip_info(chip_id_SQL)
    except:
        return print('problem')
    
    #variables
    chip_id = chip_info.get('id')
    status = chip_info.get('status')
    description = chip_info.get('description')
    
    return f"Concerning your current issue:\nChip id: \033[1m{chip_id}\033[0m\nStatus: \033[1m{status}\033[0m\nDescription: \033[1m{description}\033[0m"
    

In [46]:
connection_response('530364')

'Concerning your current issue:\nChip id: \x1bCHIP37649\x1b\nStatus: \x1binactive\x1b\nDescription: \x1bChip disabled due to non-use\x1b'